* Install JDK
* download and unpack spark
+1

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
![ ! -e "$(basename spark-2.4.1-bin-hadoop2.7.tgz)" ] && wget  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!rm -rf spark-2.4.1-bin-hadoop2.7.tgz

* install `findspark` for python to access spark
* download two datasets

*(3)*

In [0]:
!pip install -q findspark

! [ ! -e "$(basename ss.csv)" ] && wget  https://raw.githubusercontent.com/borodark/wsu/master/platforms/assignment_4/ss.csv

! [ ! -e "$(basename ch.csv)" ] && wget  https://raw.githubusercontent.com/borodark/wsu/master/platforms/assignment_4/ch.csv

--2019-04-09 12:35:16--  https://raw.githubusercontent.com/borodark/wsu/master/platforms/assignment_4/ch.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213061 (208K) [text/plain]
Saving to: ‘ch.csv’

ch.csv              100%[===================>] 208.07K  --.-KB/s    in 0.02s   

2019-04-09 12:35:17 (9.43 MB/s) - ‘ch.csv’ saved [213061/213061]



Start executing python 
* setup ENV variables for spark

*(4)*

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

import libs and start spark session

*(5)*

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# get a spark session. 
spark = SparkSession.builder.master("local[*]").getOrCreate()

* read one csv 
* create columns filter
* create new dataframe
* rename columns in new dataframe

*(7)*

In [0]:

ss_ = spark.read.csv("ss.csv", header= True, 
                      inferSchema = True)

print(ss_.columns)

columns_lambda = lambda k: k.endswith(', Current week')

common_cols = ['Reporting Area','MMWR Year','MMWR Week']

ss_cols = list(filter(columns_lambda, ss_.columns))


to_keep = common_cols + ss_cols

dfss = ss_.select(*to_keep).withColumnRenamed('Reporting Area', 'area').withColumnRenamed('MMWR Year','year').withColumnRenamed('MMWR Week','week')
dfss.columns

['Reporting Area', 'MMWR Year', 'MMWR Week', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Med', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Med, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Max', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Max, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2018', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2018, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2017', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2017, flag', 'Shiga toxin-producing Escherichia coli, Current week', 'Shiga toxin-producing Escherichia coli, Current week, flag

['area',
 'year',
 'week',
 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week',
 'Shiga toxin-producing Escherichia coli, Current week',
 'Shigellosis, Current week']

* repeat the same for another CSV

In [0]:
ch_ = spark.read.csv("ch.csv", header= True, 
                      inferSchema = True)

ch_cols = list(filter(columns_lambda, ch_.columns))


to_keep = common_cols + ch_cols

chss = ch_.select(*to_keep).withColumnRenamed('Reporting Area', 'area').withColumnRenamed('MMWR Year','year').withColumnRenamed('MMWR Week','week')

chss.columns

['area',
 'year',
 'week',
 'Chlamydia trachomatis infection, Current week',
 'Coccidioidomycosis, Current week']

* create colum list for joined dataframe
* join two dataframes on common columns: ['area', 'year', 'week']
* select only a few columns
* display schema of final dataframe

*(10)*

In [0]:
to_keep = [chss.area, chss.year, chss.week] + ss_cols + ch_cols

chss = chss.join(dfss, [chss.area == dfss.area, chss.week == dfss.week ,chss.year == dfss.year]).select(*to_keep)
chss.show(10)
chss.printSchema()

+-------------+----+----+---------------------------------------------------------------------------+----------------------------------------------------+-------------------------+---------------------------------------------+--------------------------------+
|         area|year|week|Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week|Shiga toxin-producing Escherichia coli, Current week|Shigellosis, Current week|Chlamydia trachomatis infection, Current week|Coccidioidomycosis, Current week|
+-------------+----+----+---------------------------------------------------------------------------+----------------------------------------------------+-------------------------+---------------------------------------------+--------------------------------+
|UNITED STATES|2018|   1|                                                                          9|                                                  15|                       50|                                        

* show total for each "_ , Current Week" columns by state for a year. Where are these Virgin islands anyway?

*(13)*

In [0]:
chss.groupBy(['area', 'year']).sum(*(ss_cols + ch_cols)).orderBy(['area', 'year']).show(60)

+--------------+----+--------------------------------------------------------------------------------+---------------------------------------------------------+------------------------------+--------------------------------------------------+-------------------------------------+
|          area|year|sum(Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week)|sum(Shiga toxin-producing Escherichia coli, Current week)|sum(Shigellosis, Current week)|sum(Chlamydia trachomatis infection, Current week)|sum(Coccidioidomycosis, Current week)|
+--------------+----+--------------------------------------------------------------------------------+---------------------------------------------------------+------------------------------+--------------------------------------------------+-------------------------------------+
|       ALABAMA|2018|                                                                             240|                                                       

Le's focus on Midwest: 
* Illinois 
* Indiana 
* Iowa 
* Michigan 
* Minnesota 
* Missouri 
* Ohio 
* Wisconsin

Show stats from DataFrame
Convert to pandas
Show stats

*(15)*

In [0]:
midwest = list(map( lambda k: k.upper() , ['Illinois', 'Indiana', 'Iowa', 'Michigan', 'Minnesota', 'Missouri' , 'Ohio' ,'Wisconsin' ]))

midwest_df = chss.filter( chss.area.isin(*midwest)).groupBy(['area', 'year']).sum(*(ss_cols + ch_cols)).orderBy(['area', 'year'])
midwest_df.show(8)

midwest_df.summary().show()

pd = midwest_df.toPandas()
pd.describe()
pd.info()

+---------+----+--------------------------------------------------------------------------------+---------------------------------------------------------+------------------------------+--------------------------------------------------+-------------------------------------+
|     area|year|sum(Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week)|sum(Shiga toxin-producing Escherichia coli, Current week)|sum(Shigellosis, Current week)|sum(Chlamydia trachomatis infection, Current week)|sum(Coccidioidomycosis, Current week)|
+---------+----+--------------------------------------------------------------------------------+---------------------------------------------------------+------------------------------+--------------------------------------------------+-------------------------------------+
| ILLINOIS|2018|                                                                            null|                                                     null|                 